<a href="https://colab.research.google.com/github/i-am-Aryan-Raj/Genetic-Algorithm-SMA-Backtesting/blob/main/Backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install backtesting --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import pandas as pd

# Data Import and Preprocessing

In [3]:
import yfinance as yf
data = yf.download("GOOG", start="2020-01-01", end="2021-01-01")

[*********************100%***********************]  1 of 1 completed


In [4]:
print(data.columns)

MultiIndex([('Adj Close', 'GOOG'),
            (    'Close', 'GOOG'),
            (     'High', 'GOOG'),
            (      'Low', 'GOOG'),
            (     'Open', 'GOOG'),
            (   'Volume', 'GOOG')],
           names=['Price', 'Ticker'])


In [5]:
data.reset_index(inplace=True)

In [6]:
# Rename the columns directly to avoid duplication issues
data.columns = ['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

In [7]:
data.set_index('Date', inplace=True)

In [8]:
data.head()

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2020-01-02 00:00:00+00:00,68.201012,68.368500,68.406998,67.077499,67.077499,28132000
2020-01-03 00:00:00+00:00,67.866325,68.032997,68.625000,67.277199,67.392998,23728000
2020-01-06 00:00:00+00:00,69.539726,69.710503,69.824997,67.500000,67.500000,34646000
2020-01-07 00:00:00+00:00,69.496330,69.667000,70.149498,69.518997,69.897003,30054000
2020-01-08 00:00:00+00:00,70.043991,70.216003,70.579002,69.542000,69.603996,30560000


# Backtesting using PGA approach

In [9]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

class SmaCross(Strategy):
    short_window = 10  # Short moving average window
    long_window = 20   # Long moving average window

    def init(self):
        self.short_sma = self.I(SMA, self.data.Close, self.short_window)
        self.long_sma = self.I(SMA, self.data.Close, self.long_window)

    def next(self):
        if crossover(self.short_sma, self.long_sma):
            self.buy()
        elif crossover(self.long_sma, self.short_sma):
            self.sell()


/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-cons

In [10]:
import random
from backtesting import Backtest
import numpy as np

# GA settings
population_size = 10
generations = 5
top_n = 3  # Number of top strategies to select for the next generation

def generate_random_parameters():
    return {
        'short_window': random.randint(5, 15),
        'long_window': random.randint(15, 50)
    }

# Run the GA
population = [generate_random_parameters() for _ in range(population_size)]
for gen in range(generations):
    performance = []

    for params in population:
        SmaCross.short_window = params['short_window']
        SmaCross.long_window = params['long_window']

        # Run the backtest
        bt = Backtest(data, SmaCross, cash=10000, commission=0.002)
        stats = bt.run()
        performance.append((stats['Sharpe Ratio'], params))

    # Sort and select the top strategies
    performance.sort(reverse=True, key=lambda x: x[0])
    top_strategies = performance[:top_n]

    # Display top strategies for each generation
    print(f"Generation {gen+1} top strategies:")
    for perf, params in top_strategies:
        print(f"Sharpe Ratio: {perf:.2f}, Params: {params}")

    # Mutate to create a new generation
    population = [
        {k: max(5, min(v + random.randint(-2, 2), 50)) for k, v in params.items()}
        for _, params in top_strategies
        for _ in range(int(population_size / top_n))
    ]


Generation 1 top strategies:
Sharpe Ratio: 1.12, Params: {'short_window': 10, 'long_window': 43}
Sharpe Ratio: 1.09, Params: {'short_window': 11, 'long_window': 46}
Sharpe Ratio: 1.01, Params: {'short_window': 8, 'long_window': 39}
Generation 2 top strategies:
Sharpe Ratio: 1.12, Params: {'short_window': 11, 'long_window': 43}
Sharpe Ratio: 1.12, Params: {'short_window': 11, 'long_window': 42}
Sharpe Ratio: 1.09, Params: {'short_window': 10, 'long_window': 48}
Generation 3 top strategies:
Sharpe Ratio: 1.12, Params: {'short_window': 10, 'long_window': 44}
Sharpe Ratio: 1.12, Params: {'short_window': 9, 'long_window': 44}
Sharpe Ratio: 1.12, Params: {'short_window': 11, 'long_window': 41}
Generation 4 top strategies:
Sharpe Ratio: 1.12, Params: {'short_window': 9, 'long_window': 45}
Sharpe Ratio: 1.12, Params: {'short_window': 11, 'long_window': 42}
Sharpe Ratio: 1.12, Params: {'short_window': 11, 'long_window': 41}
Generation 5 top strategies:
Sharpe Ratio: 1.12, Params: {'short_window

In [11]:
best_params = top_strategies[0][1]
SmaCross.short_window = best_params['short_window']
SmaCross.long_window = best_params['long_window']

final_backtest = Backtest(data, SmaCross, cash=10000, commission=0.002)
final_stats = final_backtest.run()
final_backtest.plot()
print(final_stats)


/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for '

Start                     2020-01-02 00:00...
End                       2020-12-31 00:00...
Duration                    364 days 00:00:00
Exposure Time [%]                   69.960474
Equity Final [$]                 13899.111887
Equity Peak [$]                  14639.670969
Return [%]                          38.991119
Buy & Hold Return [%]               28.120409
Return (Ann.) [%]                   38.810361
Volatility (Ann.) [%]               34.508177
Sharpe Ratio                         1.124671
Sortino Ratio                        2.372469
Calmar Ratio                         2.144938
Max. Drawdown [%]                  -18.093932
Avg. Drawdown [%]                   -2.985432
Max. Drawdown Duration       63 days 00:00:00
Avg. Drawdown Duration       12 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      39.052632
Worst Trade [%]                     39.052632
Avg. Trade [%]                    